In [2]:
from keras.models import Sequential,  Model
from keras.layers import Convolution2D,Input,BatchNormalization,Conv2D,Activation,Lambda,Subtract,Conv2DTranspose, PReLU
from keras.regularizers import l2
from keras.layers import  Reshape,Dense,Flatten
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import numpy as np
import math
from scipy.io import loadmat
import matplotlib.pyplot as plt
from keras.models import Sequential,  Model
from keras.layers import Convolution2D, Input, BatchNormalization , Conv2D,Activation,Lambda,Subtract,Conv2DTranspose, PReLU
import keras.backend as K
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import numpy as np
import math
from scipy import interpolate
from Model import interpolation , SRCNN_train , SRCNN_model, DNCNN_train , DNCNN_model , DNCNN_predict
#from models import interpolation , SRCNN_train , SRCNN_model, SRCNN_predict , DNCNN_train , DNCNN_model , DNCNN_predict
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [26]:
def psnr(target, ref):
    # assume RGB image
    target_data = numpy.array(target, dtype=float)
    ref_data = numpy.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(numpy.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

In [27]:
def interpolation(noisy , SNR , Number_of_pilot,interp):
    #noisy = loadmat('drive/codes/my_srcnn/My_noisy_H_%d.mat'%SNR)['My_noisy_H']
    #noisy = loadmat('drive/codes/my_srcnn/SUI5_noisy_H_%d.mat'%SNR)['SUI5_noisy_H_%d'%SNR]
    noisy_image = np.zeros((40000,72,14,2))

    noisy_image[:,:,:,0] = np.real(noisy)
    noisy_image[:,:,:,1] = np.imag(noisy)


    if (Number_of_pilot == 48):
        idx = [14*i for i in range(1, 72,6)]+[4+14*(i) for i in range(4, 72,6)]+[7+14*(i) for i in range(1, 72,6)]+[11+14*(i) for i in range(4, 72,6)]
    elif (Number_of_pilot == 16):
        idx= [4+14*(i) for i in range(1, 72,9)]+[9+14*(i) for i in range(4, 72,9)]
    elif (Number_of_pilot == 24):
        idx = [14*i for i in range(1,72,9)]+ [6+14*i for i in range(4,72,9)]+ [11+14*i for i in range(1,72,9)]
    elif (Number_of_pilot == 8):
        idx = [4+14*(i) for  i in range(5,72,18)]+[9+14*(i) for i in range(8,72,18)]
    elif (Number_of_pilot == 36):
        idx = [14*(i) for  i in range(1,72,6)]+[6+14*(i) for i in range(4,72,6)] + [11+14*i for i in range(1,72,6)]



    r = [x//14 for x in idx]
    c = [x%14 for x in idx]



    interp_noisy = np.zeros((40000,72,14,2))

    for i in range(len(noisy)):
        z = [noisy_image[i,j,k,0] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,0] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,0] = z_intp
        z = [noisy_image[i,j,k,1] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,1] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,1] = z_intp



    perfect = loadmat('C:/Users/HP/Documents/Jes PG/samsung project/Perfect_H_40000.mat')['My_perfect_H']
    #perfect = loadmat('drive/codes/my_srcnn/SUI5_perfect.mat')['SUI5_perfect_H']

    perfect_image = np.zeros((40000,72,14,2))
    perfect_image[:,:,:,0] = np.real(perfect)
    perfect_image[:,:,:,1] = np.imag(perfect)


    interp_noisy = np.concatenate((interp_noisy[:,:,:,0], interp_noisy[:,:,:,1]), axis=0).reshape(80000, 72, 14, 1)
    perfect_image = np.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(80000, 72, 14, 1)      

    #np.save('drive/codes/my_srcnn/results_22_36_rbf.npy',interp_noisy)

    #np.save('drive/codes/my_srcnn/SUI5_12_48_rbf.npy',interp_noisy)

    return interp_noisy


In [28]:
def SRCNN_model():

    input_shape = (72,14,1)
    x = Input(shape = input_shape)
    c1 = Convolution2D( 64 , 9 , 9 , activation = 'relu', kernel_initializer = 'he_normal', padding='same')(x)
    c2 = Convolution2D( 32 , 1 , 1 , activation = 'relu', kernel_initializer = 'he_normal', padding='same')(c1)
    c3 = Convolution2D( 1 , 5 , 5 , kernel_initializer = 'he_normal', padding='same')(c2)
    #c4 = Input(shape = input_shape)(c3)
    model = Model(x, c3)
    ##compile
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error']) 
    return model

In [29]:
def SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , num_pilots , SNR ):
    srcnn_model = SRCNN_model()
    print(srcnn_model.summary())

    checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]

    srcnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                    callbacks=callbacks_list, shuffle=True, epochs= 300 , verbose=0)

    #srcnn_model.save_weights("drive/codes/my_srcnn/SRCNN_SUI5_weights/SRCNN_48_12.h5")
    srcnn_model.save_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")

In [30]:
def SRCNN_predict(input_data , channel_model , num_pilots , SNR):
    srcnn_model = SRCNN_predict_model()
    srcnn_model.load_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
    predicted  = srcnn_model.predict(input_data)
    return predicted

In [31]:
def DNCNN_model ():

    inpt = Input(shape=(None,None,1))
    # 1st layer, Conv+relu
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 18 layers, Conv+BN+relu
    for i in range(18):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inpt, x)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])    
    return model

In [32]:
def DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , num_pilots , SNR ):
    dncnn_model = DNCNN_model()
    print(dncnn_model.summary())

    checkpoint = ModelCheckpoint("DNCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                               save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]

    dncnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                  callbacks=callbacks_list, shuffle=True, epochs= 200 , verbose=0)
    dncnn_model.save_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")

In [33]:
def DNCNN_predict(input_data, channel_model , num_pilots , SNR):
    dncnn_model = DNCNN_model()
    dncnn_model.load_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
    predicted  = dncnn_model.predict(input_data)
    return predicted

In [3]:
if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = 22
    Number_of_pilots = 48
    #perfect=np.zeros((40000,72,14), dtype='double')
    #noisy_input=np.zeros((40000,72,14), dtype='double')
    perfect = loadmat("C:/Users/HP/Documents/Jes PG/samsung project/Perfect_H_40000.mat")['My_perfect_H']
    noisy_input = loadmat("C:/Users/HP/Documents/Jes PG/samsung project/My_noisy_H_22.mat")["My_noisy_H"] 
    input_data = noisy_input

   # interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots)
    interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')
    

    #interp_noisy = numpy.load('drive/codes/my_srcnn/SUI5_12_48_rbf.npy')
    #perfect = loadmat('drive/codes/my_srcnn/SUI5_perfect.mat')['SUI5_perfect_H']
    perfect_image = np.zeros((len(perfect),72,14,2))
    #perfect_image[:,:,:,0] = numpy.real(perfect)
    #perfect_image[:,:,:,1] = numpy.imag(perfect)
    #perfect_image = np.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)
  

In [4]:
    perfect_image[:,:,:,0] = np.real(perfect)
    perfect_image[:,:,:,1] = np.imag(perfect)
    perfect_image = np.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)

In [5]:
    ####### ------ training SRCNN ------ #######
    idx_random = np.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
    train_data, train_label = interp_noisy[idx_random,:,:,:] , perfect_image[idx_random,:,:,:]
    #val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]    
    #SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , Number_of_pilots , SNR )
    #SRCNN_train(train_data ,train_label, val_data , val_label , channel_model , num_pilots , SNR )

In [6]:
    idx_random = np.random.rand(len(perfect_image)) < (1/9)  # uses 32000 from 36000 as training and the rest as validation
    val_data, val_label = interp_noisy[~idx_random,:,:,:] , perfect_image[~idx_random,:,:,:]  

In [38]:
    len(val_data)

71154

In [41]:
    SRCNN_train(train_data , train_label , val_data , val_label , channel_model , Number_of_pilots , SNR)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 72, 14, 1)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 2, 64)          5248      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 2, 32)          2080      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 2, 1, 1)           801       
Total params: 8,129
Trainable params: 8,129
Non-trainable params: 0
_________________________________________________________________
None


ValueError: in user code:

    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
        outputs = m

In [23]:
    ####### ------ prediction using SRCNN ------ #######
    srcnn_pred_train = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR)
    srcnn_pred_validation = SRCNN_predict(train_data, channel_model , Number_of_pilots , SNR) 

NameError: name 'SRCNN_predict_model' is not defined

In [ ]:
    ####### ------ training DNCNN ------ #######
    #DNCNN_train(input_data, channel_model , Number_of_pilots , SNR)
    DNCNN_train(train_data ,train_label, val_data , val_label, channel_model , Number_of_pilots , SNR )
    DNCNN_predict(input_data, channel_model , Number_of_pilots , SNR)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 6 640         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 6 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 6 36928       activation[0][0]                 
______________________________________________________________________________________________


Epoch 00001: val_loss improved from inf to 0.01295, saving model to DNCNN_check.h5

Epoch 00002: val_loss improved from 0.01295 to 0.01060, saving model to DNCNN_check.h5

Epoch 00003: val_loss improved from 0.01060 to 0.00898, saving model to DNCNN_check.h5

Epoch 00004: val_loss improved from 0.00898 to 0.00834, saving model to DNCNN_check.h5

Epoch 00005: val_loss improved from 0.00834 to 0.00549, saving model to DNCNN_check.h5

Epoch 00006: val_loss improved from 0.00549 to 0.00540, saving model to DNCNN_check.h5

Epoch 00007: val_loss improved from 0.00540 to 0.00519, saving model to DNCNN_check.h5

Epoch 00008: val_loss improved from 0.00519 to 0.00286, saving model to DNCNN_check.h5

Epoch 00009: val_loss did not improve from 0.00286

Epoch 00010: val_loss improved from 0.00286 to 0.00269, saving model to DNCNN_check.h5

Epoch 00011: val_loss improved from 0.00269 to 0.00222, saving model to DNCNN_check.h5

Epoch 00012: val_loss did not improve from 0.00222

Epoch 00013: val_lo

In [1]:
from keras.models import load_model
model = load_model('DNCNN_check.h5')

In [6]:
import matplotlib.pyplot as plt
plt.plot(model(model_weights))

NameError: name 'model_weights' is not defined